In [11]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv("data/Form.csv")


# Prepare data

In [13]:
# remove data not use
df = df.iloc[0:,[1,2,3]]

In [14]:
# we use age , gender and trasactions Store
df.head(5)

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."


In [15]:
# create functions to split transactions data store

def set_transactions(x):
    x = x.split(",")
    return x

In [16]:
# map values with age gender and store to format functions asscoations rule
df["Store"]  = df["Store"].apply(lambda x:set_transactions(x))


In [17]:
# map Age 
df['Age'] = df["Age"].map(lambda x:'10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+')

In [18]:
df

,Age,Gender,Store
0,10-20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
1,10-20,female,"[BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA..."
2,21-40,male,"[NAP's Coffee & Roasters, SongSarn, Yuanjai ..."
3,21-40,female,"[NAP's Coffee & Roasters, Nap x Warin, Snoop..."
4,10-20,male,"[NAP's Coffee & Roasters, sangob, Nap x Wari..."
...,...,...,...
77,10-20,female,"[NAP's Coffee & Roasters, Nap x Warin]"
78,21-40,female,"[NAP's Coffee & Roasters, sangob, SongSarn, ..."
79,21-40,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."
80,41-60,female,"[Nap x Warin, ROOF COFFEE, 11.11 Gallery and..."


In [19]:
# map age gender to create Rule
age_groups = df.groupby('Age')
age_gender_rules = {}

In [24]:
# For Loop each group
for age_group, data in age_groups:
    for gender, gender_data in data.groupby('Gender'):
        # Use the TransactionEncoder to convert the data into a suitable format for the Apriori algorithm
        te = TransactionEncoder()
        te_ary = te.fit(gender_data['Store']).transform(gender_data['Store'])
        df_temp = pd.DataFrame(te_ary, columns=te.columns_)
        prinnt
        # Run the Apriori algorithm to find frequent item sets
        frequent_itemsets = apriori(df_temp, min_support=0.2, use_colnames=True)

        # Generate association rules from the frequent item sets
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
        # Store the association rules for each age group and gender in the dictionary
        age_gender_rules[(age_group, gender)] = rules

[False False False False False False False  True  True False  True False
 False False  True False False False  True False False False False False]
[False  True False False False False False False False False  True False
 False False False  True  True False False False False  True False  True
 False False  True False]
[False False False False False False False False False False False False
 False False False  True False False False False False False False  True
 False False False False False False False False False False False False
  True False False False False]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False  True False False False False False False False
 False False  True False False False False]
[False  True  True False False  True False False False False False False
 False False False False False False False False  True]
[ True False  True  True  True False F

In [10]:
age_gender_rules.keys()

dict_keys([('10-20', 'female'), ('10-20', 'male'), ('21-40', 'female')])

In [30]:
age_gender_rules

{('10-20',
  'female'):                                           antecedents  \
 0                                           ( Amarna)   
 1                                 ( GODFATHER COFFEE)   
 2                                           ( sangob)   
 3                                       ( Bossa cafe)   
 4                           (NAP's Coffee & Roasters)   
 5                           ( MiND-K coffee and bake)   
 6                       ( LAVA  JAVA Coffee Roasters)   
 7                                (BalconyKiss Coffee)   
 8                       ( LAVA  JAVA Coffee Roasters)   
 9                                (BalconyKiss Coffee)   
 10                          ( MiND-K coffee and bake)   
 11                                          ( sangob)   
 12                          (NAP's Coffee & Roasters)   
 13                             ( sangob,  Bossa cafe)   
 14                 ( sangob, NAP's Coffee & Roasters)   
 15             ( Bossa cafe, NAP's Coffee & Roas

In [29]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,( Nap x Warin),( Amarna),0.5,1.0,0.5,1.0,1.0,0.00,inf
1,( Amarna),( Nap x Warin),1.0,0.5,0.5,0.5,1.0,0.00,1.0
2,( Phantae Coffee),( Amarna),0.5,1.0,0.5,1.0,1.0,0.00,inf
3,( Amarna),( Phantae Coffee),1.0,0.5,0.5,0.5,1.0,0.00,1.0
4,( REDCOFFEE),( Amarna),0.5,1.0,0.5,1.0,1.0,0.00,inf
...,...,...,...,...,...,...,...,...,...
777,( Amarna),"( Nap x Warin, sangob, NAP's Coffee & Roaster...",1.0,0.5,0.5,0.5,1.0,0.00,1.0
778,( sangob),"( Nap x Warin, Amarna, NAP's Coffee & Roaster...",0.5,0.5,0.5,1.0,2.0,0.25,inf
779,(NAP's Coffee & Roasters),"( Nap x Warin, Amarna, sangob, Saereesook, ...",0.5,0.5,0.5,1.0,2.0,0.25,inf
780,( Saereesook),"( Nap x Warin, Amarna, sangob, NAP's Coffee ...",0.5,0.5,0.5,1.0,2.0,0.25,inf
